# Rank Bandit Lab Walkthrough

このノートでは シナリオ読み込み → 方策実行 → 学習曲線/リグレット可視化 → ログ保存 の流れを紹介します。
詳しい CLI 操作は docs/TUTORIAL.md、シナリオ比較は scenario_gallery.ipynb、スイープ活用は sweep_comparison.ipynb も参照してください。


In [ ]:
from random import Random

from rank_bandit_lab import (
    BanditSimulator,
    CascadeEnvironment,
    SoftmaxRanking,
    plot_learning_curve,
    plot_regret_curve,
    scenario_loader,
    write_log,
)


In [ ]:
scenario = scenario_loader.load_scenario("news_headlines")
documents = [
    (entry["id"], entry["attraction"])
    for entry in scenario["documents"]
]
env_docs = [
    # Document(dataclass) を直接 import しても良いですが、ここでは dict から生成します。
    __import__("rank_bandit_lab").Document(doc_id, attraction)
    for doc_id, attraction in documents
]

In [ ]:
env = CascadeEnvironment(env_docs, slate_size=4, rng=Random(7))
policy = SoftmaxRanking([doc.doc_id for doc in env_docs], slate_size=4, temperature=0.2)
simulator = BanditSimulator(env, policy)
log = simulator.run(rounds=4000)
summary = log.summary()
summary

In [ ]:
plot_learning_curve(log)
plot_regret_curve(log)

In [ ]:
write_log("walkthrough.json", log, metadata={"scenario": "news_headlines", "algo": "softmax"})
print("Saved walkthrough.json")